In [16]:
import time
import requests

In [3]:
#PART 1
def fetch_probe_list():
  #Function to return list of active Starlink Probes
  starlink_probe_list = []
  api_endpoint = "https://atlas.ripe.net/api/v2/probes/?asn_v4=14593&status=1"

  params = {
      "asn_v4": 14593,  #Starlink ASN number
      "status": 1       #Connected Probes
  }

  response = requests.get(api_endpoint, params=params)
  json_data = response.json()

  for probe in json_data["results"]:
    starlink_probe_list.append({"id": probe["id"], "country_code": probe["country_code"]})

  print("Number of Probes: ", len(starlink_probe_list))
  return starlink_probe_list

In [5]:
probes = fetch_probe_list()
print(probes)

Number of Probes:  56
[{'id': 28, 'country_code': 'US'}, {'id': 10743, 'country_code': 'US'}, {'id': 10920, 'country_code': 'DE'}, {'id': 13040, 'country_code': 'FR'}, {'id': 17979, 'country_code': 'GB'}, {'id': 19983, 'country_code': 'AU'}, {'id': 20544, 'country_code': 'PL'}, {'id': 22802, 'country_code': 'US'}, {'id': 23127, 'country_code': 'US'}, {'id': 24742, 'country_code': 'AU'}, {'id': 26696, 'country_code': 'FR'}, {'id': 26834, 'country_code': 'CL'}, {'id': 28221, 'country_code': 'US'}, {'id': 35042, 'country_code': 'CA'}, {'id': 35681, 'country_code': 'AT'}, {'id': 35751, 'country_code': 'GB'}, {'id': 50017, 'country_code': 'US'}, {'id': 52422, 'country_code': 'US'}, {'id': 52918, 'country_code': 'AU'}, {'id': 52955, 'country_code': 'NZ'}, {'id': 53798, 'country_code': 'US'}, {'id': 54330, 'country_code': 'US'}, {'id': 55492, 'country_code': 'CA'}, {'id': 60323, 'country_code': 'DE'}, {'id': 60510, 'country_code': 'CA'}, {'id': 60929, 'country_code': 'US'}, {'id': 61081, 'cou

In [10]:
probes.sort(key= lambda a: a['country_code'])
print(probes)

countries = []
probe_list = []
for probe in probes:
    if probe['country_code'] not in countries:
        probe_list.append(probe['id'])
        countries.append(probe['country_code'])
print(probe_list)
print(countries)

[{'id': 35681, 'country_code': 'AT'}, {'id': 19983, 'country_code': 'AU'}, {'id': 24742, 'country_code': 'AU'}, {'id': 52918, 'country_code': 'AU'}, {'id': 1004453, 'country_code': 'AU'}, {'id': 1003131, 'country_code': 'BE'}, {'id': 35042, 'country_code': 'CA'}, {'id': 55492, 'country_code': 'CA'}, {'id': 60510, 'country_code': 'CA'}, {'id': 26834, 'country_code': 'CL'}, {'id': 1005623, 'country_code': 'CZ'}, {'id': 10920, 'country_code': 'DE'}, {'id': 60323, 'country_code': 'DE'}, {'id': 1001747, 'country_code': 'DE'}, {'id': 1002750, 'country_code': 'DE'}, {'id': 13040, 'country_code': 'FR'}, {'id': 26696, 'country_code': 'FR'}, {'id': 61241, 'country_code': 'FR'}, {'id': 62843, 'country_code': 'FR'}, {'id': 17979, 'country_code': 'GB'}, {'id': 35751, 'country_code': 'GB'}, {'id': 61366, 'country_code': 'GB'}, {'id': 61878, 'country_code': 'GB'}, {'id': 1005561, 'country_code': 'GB'}, {'id': 1005627, 'country_code': 'HT'}, {'id': 1004876, 'country_code': 'IT'}, {'id': 1004978, 'coun

## Credit Calculation
It costs 1 credits per ping measurement (1 packets of 48 octets). We have 56 active probes connected to Starlink. If we measure every minutes for an entire day, that is 4*24 = 96 measurements. 1 * 56 * 96 = 5376 credits

In [30]:
cur_time = time.time()


definitions = {
    "definitions": [
      {
        "type": "ping",
        "target": "google.com",
        "af": 4,
        "description": "test trial",
        "start_time": cur_time + 300,
        "end_time": cur_time + 86700,
        "interval": 900,
      }
    ],
    "probes" : [
      {"requested": 1,
       "type": "probes",
       "value": "1005623"
      }
    ]
}

def create_measurement(defs):
    api_endpoint = "https://atlas.ripe.net/api/v2/measurements"
    params={"key": '1721b7b6-100f-48b0-ad03-e2b6c95fb978'}

    x = requests.post(api_endpoint, json = defs, params=params)
    json_data = x.json()
    return json_data



In [31]:
create_measurement(definitions)

{'error': {'detail': 'There was a problem with your request',
  'status': 400,
  'title': 'Bad Request',
  'code': 102,
  'errors': [{'source': {'pointer': ''},
    'detail': 'We do not allow more than 25 concurrent measurements to the same target: google.com.'}]}}

In [14]:
def get_results(id):
    api_endpoint = "https://atlas.ripe.net/api/v2/measurements/" + id + "/results"
    x = requests.get(api_endpoint)
    json_data = x.json()
    return json_data

In [15]:
get_results(1)

[{'fw': 5080,
  'mver': '2.6.2',
  'lts': 6,
  'dst_name': '142.250.179.206',
  'af': 4,
  'dst_addr': '142.250.179.206',
  'src_addr': '192.168.10.88',
  'proto': 'ICMP',
  'ttl': 113,
  'size': 64,
  'result': [{'rtt': 34.045007}, {'rtt': 40.057548}, {'rtt': 41.63382}],
  'dup': 0,
  'rcvd': 3,
  'sent': 3,
  'min': 34.045007,
  'max': 41.63382,
  'avg': 38.5787916667,
  'msm_id': 52727092,
  'prb_id': 10920,
  'timestamp': 1682631567,
  'msm_name': 'Ping',
  'from': '145.224.73.88',
  'type': 'ping',
  'group_id': 52727092,
  'step': None,
  'stored_timestamp': 1682631567},
 {'fw': 5080,
  'mver': '2.6.2',
  'lts': 9,
  'dst_name': '142.250.179.206',
  'af': 4,
  'dst_addr': '142.250.179.206',
  'src_addr': '192.168.1.110',
  'proto': 'ICMP',
  'ttl': 58,
  'size': 64,
  'result': [{'rtt': 52.819345}, {'rtt': 44.352708}, {'rtt': 43.667051}],
  'dup': 0,
  'rcvd': 3,
  'sent': 3,
  'min': 43.667051,
  'max': 52.819345,
  'avg': 46.946368,
  'msm_id': 52727092,
  'prb_id': 13040,
  't

In [30]:
time.time() 

1682633345.662287